In [1]:
import numpy as np
from scipy.optimize import minimize
from cvxopt import matrix, solvers

In [2]:
#q2a

def objective_function(params, y, lamb, K):
    n = len(y)
    
    alpha = params[:n]
    alpha0 = params[n]
    
    linear_combination = K @ alpha + alpha0
    loss = np.log(1 + np.exp(-y * linear_combination)).sum()
    
    regularization = (lamb / 2) * np.dot(alpha.T, K @ alpha)
    return loss + regularization


def adjBinDev(X, y, lamb, kernel_func):
    n, d = X.shape
    K = kernel_func(X, X)
    initial_params = np.zeros(n + 1)
    
    result = minimize(objective_function, initial_params, args=(y, lamb, K))
    
    alpha = result.x[:-1]
    alpha0 = result.x[-1]

    return alpha, alpha0



#  usage
X = np.random.randn(100, 1) 
y = np.random.choice([-1, 1], size=100) 
lamb = 0.1  
kernel_func = lambda X1, X2: np.dot(X1, X2.T)

alpha, alpha_0 = adjBinDev(X, y, lamb,kernel_func)

print("Optimized weights:", alpha)
print("Optimized bias:", alpha_0) 

Optimized weights: [-7.09964362e-04  2.00693483e-03  4.00528389e-04 -8.93936168e-04
 -1.64868969e-03 -7.39176129e-04 -6.32252229e-04 -1.08744419e-04
 -1.75948432e-03  5.34773251e-05  1.35566747e-03 -1.21086547e-04
 -5.83700890e-04 -2.70887617e-04  1.79322877e-04  2.17045415e-05
 -6.28074055e-05 -2.77006856e-04 -7.13888728e-04  2.24524190e-03
  4.22616668e-04  2.57533198e-04  1.29573331e-03 -9.40138934e-04
  5.12852322e-04 -8.12083216e-04  2.46673723e-04 -3.81270970e-04
 -6.12864709e-04 -9.72989544e-04  2.42738030e-04  2.39190231e-04
  1.39906750e-04  2.32557070e-06  1.43955166e-05 -5.08877648e-04
 -1.25178966e-04  2.56142126e-06 -6.97838973e-04 -2.26258196e-04
 -1.31768020e-03 -5.50901326e-04  4.96650067e-04 -4.08167195e-04
  1.09934412e-03  1.05993720e-03  3.67392023e-04 -2.20704169e-04
 -1.23063533e-04 -1.25939433e-04  4.37605116e-04 -1.47797560e-04
  1.41588960e-03  1.27293804e-03  1.75989025e-03  1.85364496e-03
  1.01786239e-03  2.44323068e-03  1.52930079e-03 -1.83718758e-03
  1.14

In [3]:
#q2b

def adjHinge(X,y,lamb,kernel_func,stabilizer=1e-5):
    n = len(y)
    K = kernel_func(X,X)  # Compute the kernel matrix
    y = np.array(y, dtype=np.double)

    # Create the P matrix
    P = np.zeros((n + 1 + n, n + 1 + n))  # α, α_0, ξ
    P[:n, :n] = K  # Kernel matrix for α terms
    #P = P * np.outer(y, y)  # Modulate by outer product of labels
    P += stabilizer * np.eye(n + 1 + n)  # Stabilization

    #1 vector
    q = np.hstack([np.zeros(n + 1), lamb * np.ones(n)])
    
    #G matrix
    G = np.zeros((2 * n, n + 1 + n))
    
    G[:n, n+1:] = np.eye(n)  # For ξ ≥ 0
    G[n:, :n] = -K * y[:, None]  # -Δ(y)K part of the hinge constraint
    G[n:, n] = -y  # -Δ(y)α_0 part
    G[n:, n+1:] = np.eye(n)  #slack variables
    
    # Create the h vector
    h = np.hstack([np.zeros(n), -np.ones(n)])  # 0s for ξ ≥ 0 and 1s for the hinge constraint

    # Convert everything to CVXOPT matrices for solving
    P = matrix(P)
    q = matrix(q)
    G = matrix(G)
    h = matrix(h)
  

    #Solve
    solvers.options['show_progress'] = False
    solution = solvers.qp(P, q, G, h)

    # Extract solutions
    alphas = np.array(solution['x'][:n])
    alpha0 = np.array(solution['x'][n])

    return alphas, alpha0

# Example usage
X = np.random.randn(100, 2)  # 100 samples, 2 features each
y = np.random.choice([-1, 1], size=100)  # Binary labels
lamb = 1.0  # Regularization parameter

def kernel_function(X):
    """ Linear kernel as a placeholder. Replace with appropriate kernel logic. """
    return np.dot(X, X.T)

a, a0 = adjHinge(X, y, lamb,kernel_func)

print("Optimized weights:", a.flatten())
print("Optimized bias:", a0)

Optimized weights: [ 6.74287837e-09 -1.75820952e-09 -5.77008761e-09 -3.26955109e-09
 -4.94406363e-09 -1.82373385e-09  1.80029279e-09  1.14533156e-11
  1.54705781e-09  2.08002916e-09  2.52541622e-09 -3.56459261e-09
  6.54447014e-10  1.20782754e-09 -9.67435626e-10 -9.44198744e-10
 -3.53941413e-09  2.07095775e-09 -3.90815994e-09  1.22561235e-09
 -9.35842782e-10  2.41222302e-09 -1.65670732e-09  5.31345487e-09
 -3.12635984e-10  3.08651572e-10  7.35471669e-10 -9.14629054e-10
 -3.93122264e-10  3.92756280e-11  2.61985870e-09 -5.74765091e-10
  3.43095961e-09  3.21307938e-10 -1.79793766e-09  5.06616819e-09
  4.65303804e-09 -2.75315663e-09  2.29134850e-09 -3.65314968e-09
  1.08950663e-09  4.25818633e-10 -6.80929807e-10  1.83449699e-09
 -1.92140949e-09  1.18973707e-09 -3.92927817e-09  6.61153146e-10
 -3.71508645e-09 -6.06382782e-10 -1.18369897e-09 -1.11468199e-09
  4.82950768e-09 -1.21067833e-09  2.74077622e-09 -1.01242778e-09
  2.14670451e-10  1.40083794e-09  9.04213072e-10 -5.99678671e-09
  9.96

In [4]:
#q2c

def adjClassify(Xtest, a, a0, X, kernel_func):



SyntaxError: expected ':' (928105013.py, line 3)